In [2]:
import pandas as pd
import numpy as np
import os.path
import importlib
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from dateutil.relativedelta import relativedelta

import utilities.variables as variables
import utilities.data_ticker_service as data_ticker_service
importlib.reload(data_ticker_service)
importlib.reload(variables)

<module 'utilities.variables' from '/Users/herbishtini/Documents/UNI/Master Thesis/sustainability_portfolio_optimisation/utilities/variables.py'>

### Reading file

In [12]:
df = pd.read_csv('../data/data_1_esg_raw.csv') 
df.head(5)

,company_name,ticker_symbol,company_esg_score,company_esg_score_group,industry
0,HENSOLDT AG,ETR:HAG,14.9,Low ESG Risk,Aerospace & Defense
1,Aptiv Plc,NYS:APTV,9.1,Negligible ESG Risk,Auto Components
2,"BorgWarner, Inc.",NYS:BWA,10.0,Negligible ESG Risk,Auto Components
3,Bosch Fren Sistemleri Sanayi ve Ticaret AS,IST:BFREN.E,8.1,Negligible ESG Risk,Auto Components
4,Bosch Ltd.,BOM:500530,6.5,Negligible ESG Risk,Auto Components


### Sort by ESG score

In [13]:
df_sorted = df.sort_values('company_esg_score')
df_sorted

,company_name,ticker_symbol,company_esg_score,company_esg_score_group,industry
4184,ROADIS Transportation Holding SLU,-,4.0,Negligible ESG Risk,Transportation Infrastructure
1018,JAB Holding Co. SARL,-,4.2,Negligible ESG Risk,Diversified Financials
2037,Dexus,ASX:DXS,4.2,Negligible ESG Risk,Real Estate
4180,Entidad Pública Empresarial ADIF-Alta Velocidad,-,4.3,Negligible ESG Risk,Transportation Infrastructure
3498,RS Group Plc,LON:RS1,4.5,Negligible ESG Risk,Technology Hardware
...,...,...,...,...,...
1428,International Finance Facility for Immunisatio...,-,20.0,Low ESG Risk,Healthcare
3843,Orange SA,PAR:ORA,20.0,Low ESG Risk,Telecommunication Services
4365,Promigas SA ESP,BOG:PROMIGAS,20.0,Low ESG Risk,Utilities
3950,Li & Fung Ltd.,-,20.0,Low ESG Risk,Textiles & Apparel


In [14]:
len(df_sorted)

4386

### Group by industry and company-esg-score-group

In [15]:
# Group by the first and second columns and count the occurrences
df_grouped = df.groupby(['industry', 'company_esg_score_group']).size().reset_index(name='Count')
df_grouped = df_grouped.sort_values(by=['industry', 'company_esg_score_group'], ascending=[True, False])
df_grouped

,industry,company_esg_score_group,Count
0,Aerospace & Defense,Low ESG Risk,1
2,Auto Components,Negligible ESG Risk,12
1,Auto Components,Low ESG Risk,105
3,Automobiles,Low ESG Risk,22
5,Banks,Negligible ESG Risk,43
...,...,...,...
63,Transportation,Low ESG Risk,107
65,Transportation Infrastructure,Negligible ESG Risk,21
64,Transportation Infrastructure,Low ESG Risk,92
67,Utilities,Negligible ESG Risk,11


### Remove stocks without a valid stock exchange

In [16]:
len(df_sorted[df_sorted['ticker_symbol'] != '-'])

3801

In [17]:
df_sorted[df_sorted['ticker_symbol'] != '-']

,company_name,ticker_symbol,company_esg_score,company_esg_score_group,industry
2037,Dexus,ASX:DXS,4.2,Negligible ESG Risk,Real Estate
3498,RS Group Plc,LON:RS1,4.5,Negligible ESG Risk,Technology Hardware
3486,"Kimball Electronics, Inc.",NAS:KE,4.5,Negligible ESG Risk,Technology Hardware
2100,TAG Immobilien AG,ETR:TEG,4.6,Negligible ESG Risk,Real Estate
2105,Unibail-Rodamco-Westfield SE,PAR:URW,4.7,Negligible ESG Risk,Real Estate
...,...,...,...,...,...
1140,Nippon Life India Asset Management Ltd.,BOM:540767,20.0,Low ESG Risk,Diversified Financials
3850,"Taiwan Mobile Co., Ltd.",TAI:3045,20.0,Low ESG Risk,Telecommunication Services
4160,Union Pacific Corp.,NYS:UNP,20.0,Low ESG Risk,Transportation
3843,Orange SA,PAR:ORA,20.0,Low ESG Risk,Telecommunication Services


### Create new columns "stock_exchange" & "stock_ticker_symbol"

In [18]:
# Create new column stock_exchange by splitting ticker_symbol
df_sorted['stock_exchange'] = df_sorted['ticker_symbol'].str.split(':').str[0]

In [19]:
# Create new column stock_ticker_symbol by splitting ticker_symbol
df_sorted['stock_ticker_symbol'] = df_sorted['ticker_symbol'].str.split(':').str[1]

In [20]:
df_sorted[df_sorted['ticker_symbol'] != '-']['stock_exchange'].value_counts('')

stock_exchange
NYS    551
NAS    408
TKS    372
LON    226
TAI    183
      ... 
LIM      1
LIT      1
BRA      1
FRA      1
BER      1
Name: count, Length: 62, dtype: int64

## Selected Stock-Exchanges

In [21]:
len(df_sorted)

4386

### Filter Stock Exchanges

In [22]:
# drop existing column "stock_ticker_symbol"
df_sorted = df_sorted.drop(columns=['ticker_symbol'])

In [23]:
# Filter by a single column value 
df_filtered = df_sorted[df_sorted['stock_exchange'].isin(['NYS', 'NAS', 'TKS', 'LON', 'ETR'])]
len(df_filtered)

1662

In [24]:
df_filtered

,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol
3498,RS Group Plc,4.5,Negligible ESG Risk,Technology Hardware,LON,RS1
3486,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE
2100,TAG Immobilien AG,4.6,Negligible ESG Risk,Real Estate,ETR,TEG
2068,LEG Immobilien SE,5.1,Negligible ESG Risk,Real Estate,ETR,LEG
521,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS
...,...,...,...,...,...,...
3676,"KVH Industries, Inc. (Delaware)",20.0,Low ESG Risk,Technology Hardware,NAS,KVHI
2905,Moonpig Group Plc,20.0,Low ESG Risk,Retailing,LON,MOON
1963,"NeoGenomics, Inc.",20.0,Low ESG Risk,Pharmaceuticals,NAS,NEO
3057,"Japan Material Co., Ltd.",20.0,Low ESG Risk,Semiconductors,TKS,6055


#### Group by industry and company-esg-score-group

In [25]:
len(df_filtered)

1662

In [26]:
# Group by the first and second columns and count the occurrences
df_filtered_grouped = df_filtered.groupby(['industry', 'company_esg_score_group']).size().reset_index(name='Count')
df_filtered_grouped = df_filtered_grouped.sort_values(by=['industry', 'company_esg_score_group'], ascending=[True, False])
df_filtered_grouped

,industry,company_esg_score_group,Count
0,Aerospace & Defense,Low ESG Risk,1
2,Auto Components,Negligible ESG Risk,5
1,Auto Components,Low ESG Risk,45
3,Automobiles,Low ESG Risk,4
5,Banks,Negligible ESG Risk,1
4,Banks,Low ESG Risk,15
6,Building Products,Low ESG Risk,13
7,Chemicals,Low ESG Risk,11
9,Commercial Services,Negligible ESG Risk,32
8,Commercial Services,Low ESG Risk,83


In [27]:
df_filtered.to_csv('../data/data_2_esg_filtered.csv')

### Adding market capital of companies

In [28]:
if 'df_market_cap' not in locals():
    df_market_cap = pd.read_csv('../data/data_2_esg_filtered.csv')
df_market_cap

,Unnamed: 0,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol
0,3498,RS Group Plc,4.5,Negligible ESG Risk,Technology Hardware,LON,RS1.L
1,3486,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE
2,2100,TAG Immobilien AG,4.6,Negligible ESG Risk,Real Estate,ETR,TEG.DE
3,2068,LEG Immobilien SE,5.1,Negligible ESG Risk,Real Estate,ETR,LEG.DE
4,521,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS
...,...,...,...,...,...,...,...
1657,3676,"KVH Industries, Inc. (Delaware)",20.0,Low ESG Risk,Technology Hardware,NAS,KVHI
1658,2905,Moonpig Group Plc,20.0,Low ESG Risk,Retailing,LON,MOON.L
1659,1963,"NeoGenomics, Inc.",20.0,Low ESG Risk,Pharmaceuticals,NAS,NEO
1660,3057,"Japan Material Co., Ltd.",20.0,Low ESG Risk,Semiconductors,TKS,6055.T


#### Updating `stock_ticker_symbol` for Frankfurt and London
Frankfurt & London tickers require a postfix '.DE' & '.L' respectively for Yahoo API to recognize them.

In [29]:
# Postfix to add
frankfurt_postfix = '.DE'
london_postfix = '.L'
tokyo_postfix = '.T'

# Condition: Add postfix '.DE' to 'stock_exchange' column if the 'stock_ticker_symbol' column is 'ETR' (Frankfurt)
df_market_cap['stock_ticker_symbol'] = np.where((df_market_cap['stock_exchange'] == 'ETR'), df_market_cap['stock_ticker_symbol'] + frankfurt_postfix, df_market_cap['stock_ticker_symbol'])

# Condition: Add postfix '.L' to 'stock_exchange' column if the 'stock_ticker_symbol' column is 'Lon' (London)
df_market_cap['stock_ticker_symbol'] = np.where((df_market_cap['stock_exchange'] == 'LON'), df_market_cap['stock_ticker_symbol'] + london_postfix, df_market_cap['stock_ticker_symbol'])

# Condition: Add postfix '.T' to 'stock_exchange' column if the 'stock_ticker_symbol' column is 'TKS' (Tokyo)
df_market_cap['stock_ticker_symbol'] = np.where((df_market_cap['stock_exchange'] == 'TKS'), df_market_cap['stock_ticker_symbol'] + tokyo_postfix, df_market_cap['stock_ticker_symbol'])

In [30]:
df_market_cap[['stock_exchange', 'stock_ticker_symbol']]

,stock_exchange,stock_ticker_symbol
0,LON,RS1.L.L
1,NAS,KE
2,ETR,TEG.DE.DE
3,ETR,LEG.DE.DE
4,NYS,SCS
...,...,...
1657,NAS,KVHI
1658,LON,MOON.L.L
1659,NAS,NEO
1660,TKS,6055.T.T


## Fetch Market-Capital

In [31]:
# Fetch market cap in batches of 50 with a 3-second delay between batches
if os.path.isfile('market_caps.csv'):
    df_market_caps = data_ticker_service.fetch_market_cap(df_market_cap['stock_ticker_symbol'], batch_size=50, delay=3)

df_market_caps


      Unnamed: 0 stock_ticker_symbol  market_capital
0              0               RS1.L    3.723188e+09
1              1                  KE    5.958660e+08
2              2              TEG.DE    2.539239e+09
3              3              LEG.DE    6.009704e+09
4              4                 SCS    1.588731e+09
...          ...                 ...             ...
1657        1657                KVHI    9.148317e+07
1658        1658              MOON.L    7.034287e+08
1659        1659                 NEO    1.979536e+09
1660        1660              6055.T    2.270510e+11
1661        1661                 UNP    1.488027e+11

[1662 rows x 3 columns]


In [32]:
df_market_caps.to_csv('../data/data_3_market_caps.csv')

In [33]:
if 'df_market_caps' not in locals():
    df_market_caps = pd.read_csv('../data/data_3_market_caps.csv', index_col=0)

In [39]:
df_market_caps = df_market_caps[['stock_ticker_symbol', 'market_capital']]
df_market_caps

,stock_ticker_symbol,market_capital
0,RS1.L,3.723188e+09
1,KE,5.958660e+08
2,TEG.DE,2.539239e+09
3,LEG.DE,6.009704e+09
4,SCS,1.588731e+09
...,...,...
1657,KVHI,9.148317e+07
1658,MOON.L,7.034287e+08
1659,NEO,1.979536e+09
1660,6055.T,2.270510e+11


Merging ESG score with market-capital of companies

In [45]:
df_market_cap = pd.merge(df_market_cap, df_market_caps, on='stock_ticker_symbol', left_index=False, right_index=False)
df_market_cap

,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol,market_capital
0,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE,5.958660e+08
1,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS,1.588731e+09
2,HNI Corp.,5.4,Negligible ESG Risk,Commercial Services,NYS,HNI,2.324179e+09
3,"Avnet, Inc.",5.6,Negligible ESG Risk,Technology Hardware,NAS,AVT,4.910356e+09
4,ACCO Brands Corp.,5.7,Negligible ESG Risk,Commercial Services,NYS,ACCO,4.781985e+08
...,...,...,...,...,...,...,...
954,"Halozyme Therapeutics, Inc.",19.9,Low ESG Risk,Pharmaceuticals,NAS,HALO,6.968251e+09
955,Matthews International Corp.,20.0,Low ESG Risk,Commercial Services,NAS,MATW,8.512784e+08
956,"KVH Industries, Inc. (Delaware)",20.0,Low ESG Risk,Technology Hardware,NAS,KVHI,9.148317e+07
957,"NeoGenomics, Inc.",20.0,Low ESG Risk,Pharmaceuticals,NAS,NEO,1.979536e+09


### Market Capital to Euro

To make sense of the market-capital value we have to first pick a preferred currency and convert them all to it.
For this purpose we will be using Euro as the standard currency.


In [54]:
df_market_cap['market_capital_euro'] = df_market_cap['market_capital']

Exchange Rate (19.08.2024)

In [57]:
exchange_rate = {
    "yen_to_euro": 0.0058,
    "us_to_euro": 0.92,
    "pound_to_euro": 1.19
}

In [69]:
for i, row in enumerate(df_market_cap):
    # Yen
    if df_market_cap['stock_exchange'][i] == 'TKS':
        df_market_cap.at[i, 'market_capital_euro'] = df_market_cap['market_capital'][i] * exchange_rate['pound_to_euro']
    # Dollar
    if df_market_cap['stock_exchange'][i] == 'NYS':
        df_market_cap.at[i, 'market_capital_euro'] = df_market_cap['market_capital'][i] * exchange_rate['us_to_euro']
    if df_market_cap['stock_exchange'][i] == 'NAS':
        df_market_cap.at[i, 'market_capital_euro'] = df_market_cap['market_capital'][i] * exchange_rate['us_to_euro']
    # Pound
    if df_market_cap['stock_exchange'][i] == 'LON':
        df_market_cap.at[i, 'market_capital_euro'] = df_market_cap['market_capital'][i] * exchange_rate['pound_to_euro']

                        company_name  company_esg_score  \
0          Kimball Electronics, Inc.                4.5   
1                    Steelcase, Inc.                5.3   
2                          HNI Corp.                5.4   
3                        Avnet, Inc.                5.6   
4                  ACCO Brands Corp.                5.7   
..                               ...                ...   
954      Halozyme Therapeutics, Inc.               19.9   
955     Matthews International Corp.               20.0   
956  KVH Industries, Inc. (Delaware)               20.0   
957                NeoGenomics, Inc.               20.0   
958              Union Pacific Corp.               20.0   

    company_esg_score_group             industry stock_exchange  \
0       Negligible ESG Risk  Technology Hardware            NAS   
1       Negligible ESG Risk  Commercial Services            NYS   
2       Negligible ESG Risk  Commercial Services            NYS   
3       Negligible ESG 

In [72]:
df_market_cap.to_csv('../data/data_4_market_cap_euro.csv')
df_market_cap

,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol,market_capital,market_capital_euro
0,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE,5.958660e+08,5.481967e+08
1,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS,1.588731e+09,1.461633e+09
2,HNI Corp.,5.4,Negligible ESG Risk,Commercial Services,NYS,HNI,2.324179e+09,2.138245e+09
3,"Avnet, Inc.",5.6,Negligible ESG Risk,Technology Hardware,NAS,AVT,4.910356e+09,4.517528e+09
4,ACCO Brands Corp.,5.7,Negligible ESG Risk,Commercial Services,NYS,ACCO,4.781985e+08,4.399426e+08
...,...,...,...,...,...,...,...,...
954,"Halozyme Therapeutics, Inc.",19.9,Low ESG Risk,Pharmaceuticals,NAS,HALO,6.968251e+09,6.410791e+09
955,Matthews International Corp.,20.0,Low ESG Risk,Commercial Services,NAS,MATW,8.512784e+08,7.831761e+08
956,"KVH Industries, Inc. (Delaware)",20.0,Low ESG Risk,Technology Hardware,NAS,KVHI,9.148317e+07,8.416451e+07
957,"NeoGenomics, Inc.",20.0,Low ESG Risk,Pharmaceuticals,NAS,NEO,1.979536e+09,1.821173e+09


### Column normalization

Since ESG-Score and market-capital are on different scales, it's important to normalize them so they can be compared directly.

In [11]:
df_scaled = pd.read_csv('../data/data_4_market_cap_euro.csv')

In [12]:
# Normalize ESG-Score and market-capital
scaler = MinMaxScaler()
df_scaled[['company_esg_score_scale', 'market_capital_scale']] = scaler.fit_transform(df_scaled[['company_esg_score', 'market_capital_euro']])

In [13]:
df_scaled

,Unnamed: 0,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol,market_capital,market_capital_euro,company_esg_score_scale,market_capital_scale
0,0,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE,5.958660e+08,5.481967e+08,0.000000,0.000159
1,1,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS,1.588731e+09,1.461633e+09,0.051613,0.000442
2,2,HNI Corp.,5.4,Negligible ESG Risk,Commercial Services,NYS,HNI,2.324179e+09,2.138245e+09,0.058065,0.000652
3,3,"Avnet, Inc.",5.6,Negligible ESG Risk,Technology Hardware,NAS,AVT,4.910356e+09,4.517528e+09,0.070968,0.001389
4,4,ACCO Brands Corp.,5.7,Negligible ESG Risk,Commercial Services,NYS,ACCO,4.781985e+08,4.399426e+08,0.077419,0.000126
...,...,...,...,...,...,...,...,...,...,...,...
954,954,"Halozyme Therapeutics, Inc.",19.9,Low ESG Risk,Pharmaceuticals,NAS,HALO,6.968251e+09,6.410791e+09,0.993548,0.001975
955,955,Matthews International Corp.,20.0,Low ESG Risk,Commercial Services,NAS,MATW,8.512784e+08,7.831761e+08,1.000000,0.000232
956,956,"KVH Industries, Inc. (Delaware)",20.0,Low ESG Risk,Technology Hardware,NAS,KVHI,9.148317e+07,8.416451e+07,1.000000,0.000016
957,957,"NeoGenomics, Inc.",20.0,Low ESG Risk,Pharmaceuticals,NAS,NEO,1.979536e+09,1.821173e+09,1.000000,0.000554


To create a final score, we use the ESG score and the market capital score, applying the following weights: the ESG score is multiplied by a coefficient of 0.25, and the market capital score is multiplied by a coefficient of 0.75.

In [14]:
# Create final score
coefficient_esg_score = 0.25
coefficient_market_capital = 0.75
df_scaled['score'] = df_scaled['company_esg_score_scale'] * coefficient_esg_score + df_scaled['market_capital_scale'] * coefficient_market_capital

# Sort by score
df_scaled.sort_values(by='score', ascending=False)

,Unnamed: 0,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol,market_capital,market_capital_euro,company_esg_score_scale,market_capital_scale,score
560,560,"Apple, Inc.",16.8,Low ESG Risk,Technology Hardware,NAS,AAPL,3.509669e+12,3.228895e+12,0.793548,1.000000,0.948387
282,282,Microsoft Corp.,14.2,Low ESG Risk,Software & Services,NAS,MSFT,3.296378e+12,3.032668e+12,0.625806,0.939227,0.860872
217,217,NVIDIA Corp.,13.2,Low ESG Risk,Semiconductors,NAS,NVDA,2.902354e+12,2.670165e+12,0.561290,0.826958,0.760541
821,821,Broadcom Inc.,18.9,Low ESG Risk,Semiconductors,NAS,AVGO,7.260681e+11,6.679827e+11,0.929032,0.206868,0.387409
584,584,"UnitedHealth Group, Inc.",17.0,Low ESG Risk,Healthcare,NYS,UNH,5.276384e+11,4.854273e+11,0.806452,0.150330,0.314360
...,...,...,...,...,...,...,...,...,...,...,...,...
1,1,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS,1.588731e+09,1.461633e+09,0.051613,0.000442,0.013235
0,0,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE,5.958660e+08,5.481967e+08,0.000000,0.000159,0.000120
594,594,Triton International Ltd.,17.1,Low ESG Risk,Commercial Services,NYS,TRTN.PRE,NaN,NaN,0.812903,NaN,NaN
645,645,National Rural Utilities Cooperative Finance C...,17.6,Low ESG Risk,Banks,NYS,NRUC,NaN,NaN,0.845161,NaN,NaN


In [18]:
columns_relevant = ['company_name', 'industry', 'stock_exchange', 'stock_ticker_symbol', 'market_capital_euro', 'score']
df_scaled[columns_relevant].to_csv('../data/data_5_scaled.csv')

## Historical returns
Historical returns are analyzed over the past 25 years. This timeframe is chosen because it captures both short-term volatility and long-term market disruptions, while filtering out the daily or monthly fluctuations.

In [20]:
df_returns = pd.read_csv('../data/data_5_scaled.csv', index_col=0)

In [21]:
df_returns.head()

,company_name,industry,stock_exchange,stock_ticker_symbol,market_capital_euro,score
0,"Kimball Electronics, Inc.",Technology Hardware,NAS,KE,5.481967e+08,0.000120
1,"Steelcase, Inc.",Commercial Services,NYS,SCS,1.461633e+09,0.013235
2,HNI Corp.,Commercial Services,NYS,HNI,2.138245e+09,0.015005
3,"Avnet, Inc.",Technology Hardware,NAS,AVT,4.517528e+09,0.018784
4,ACCO Brands Corp.,Commercial Services,NYS,ACCO,4.399426e+08,0.019449


In [22]:
tickers = df_returns['stock_ticker_symbol']
# Format the new date as 'YYYY-MM-DD'
start_date = (datetime.now() - relativedelta(years=variables.max_span_years)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')
#
#monthly_returns = data_ticker_service.get_monthly_returns(tickers, start_date, end_date)
df_monthly_adj_close = data_ticker_service.get_returns_in_chunks(tickers, start_date, end_date, interval='1mo', chunk_size=20, sleep_duration=5)
df_monthly_adj_close

1      SCS
2      HNI
3      AVT
4     ACCO
5     KEYS
6     CBRE
7      BRC
8     PGRE
9      BHE
10    PLXS
11     HAS
12    CHGG
13     JLL
14    OLED
15     CDW
16    FLEX
17    REZI
18     TPL
19     ARW
Name: stock_ticker_symbol, dtype: object
Pausing to avoid overloading the API...
21    NOVT
22     RYN
23     SNX
24    CMPR
25    TTMI
26     PBI
27    ASGN
28    HOUS
29     VVV
30     AVB
31    STWD
32     KFY
33     MEI
34    RGLD
35    KFRC
36    SANM
37    YETI
38     ACN
39     HPP
Name: stock_ticker_symbol, dtype: object
Pausing to avoid overloading the API...
41      CTS
42      IPG
43      BMI
44     SCSC
45      MAN
46      PDM
47    KELYA
48       ST
49      CCK
50     APTV
51     ITRI
52      WDC
53      EBF
54     TNET
55     MLKN
56      JBL
57      LXP
58     TRMB
59      NLY
Name: stock_ticker_symbol, dtype: object
Pausing to avoid overloading the API...
61    SBAC
62     KRG
63     ATR
64    ELME
65    ESRT
66    CGNX
67    ZBRA
68     DLX
69     BWA
70     ELV
7


1 Failed download:
['TRTN.PRE']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


Pausing to avoid overloading the API...
601    ARHS
602     HZO
603    MEDP
604    BKNG
605     AMN
606    ROST
607     CAL
608    MANH
609     EQH
610    EWCZ
611     HTZ
612     TFC
613     OHI
614    TROW
615    FSLR
616    NEOG
617     DAY
618    SNDR
619     MDB
Name: stock_ticker_symbol, dtype: object
Pausing to avoid overloading the API...
621    MERC
622    SHYF
623    CHEF
624    HCSG
625      WU
626    APLE
627    MSGS
628     DBI
629     ROK
630    GNTX
631    WDAY
632     CCS
633    INFA
634    COHR
635    CATO
636      RH
637    LSCC
638    CHRW
639     IVR
Name: stock_ticker_symbol, dtype: object
Pausing to avoid overloading the API...
641    ONTF
642      ZD
643    CRWD
644    SLAB
645    NRUC
646     GTN
647    SAIA
648      KN
649    LINC
650    DAKT
651    CHCT
652     PRI
653    CHKP
654    HFFG
655    ALIT
656    TEAM
657    DGII
658    EVTC
659     AFL
Name: stock_ticker_symbol, dtype: object
Pausing to avoid overloading the API...
661    PECO
662    NCNO
663     S


1 Failed download:
['LGF.A']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


Pausing to avoid overloading the API...
721    MNDY
722    EVRI
723    HSTM
724     DNB
725     AXP
726     ORC
727    ARRY
728     FOR
729     GEF
730    EHTH
731    TPIC
732    NTST
733     PKG
734      OC
735     ARR
736    IMAX
737     IHS
738     TAL
739    PAGS
Name: stock_ticker_symbol, dtype: object
Pausing to avoid overloading the API...
741    FLGT
742    WRBY
743    PGNY
744     UTI
745    BEPC
746      DV
747    WINA
748     GTY
749     TYL
750     PRG
751     ADV
752     BLK
753    NTCT
754    PDCO
755    PAYC
756    GDDY
757    RMBS
758    PACB
759    MGRC
Name: stock_ticker_symbol, dtype: object
Pausing to avoid overloading the API...
761      MSA
762       RC
763     ELAN
764     WTRG
765     UPWK
766      KBR
767     ADUS
768     PRPL
769     PWSC
770      RSG
771      PTC
772     ALGN
773      CRH
774      PRU
775      CNK
776      LNC
777      OUT
778    LBRDK
779      CTO
Name: stock_ticker_symbol, dtype: object
Pausing to avoid overloading the API...
781      MU
78

,KE,SCS,HNI,AVT,ACCO,KEYS,CBRE,BRC,PGRE,BHE,...,ENPH,RUN,SUP,MATX,KIDS,HALO,MATW,KVHI,NEO,UNP
Date,,,,,,,,,,,,,,,,,,,,,
1999-08-01,NaN,6.013282,11.762950,17.141674,NaN,NaN,NaN,8.704611,NaN,13.926089,...,NaN,NaN,15.431546,6.066541,NaN,NaN,9.999043,2.43750,NaN,7.509912
1999-09-01,NaN,5.658068,9.648345,16.270067,NaN,NaN,NaN,9.284917,NaN,13.358641,...,NaN,NaN,15.431546,6.040688,NaN,NaN,11.104928,2.37500,NaN,7.413507
1999-10-01,NaN,5.165832,9.836304,21.209169,NaN,NaN,NaN,8.414455,NaN,6.052765,...,NaN,NaN,14.708194,6.072314,NaN,NaN,9.261786,3.06250,NaN,8.634527
1999-11-01,NaN,5.319273,10.838723,21.354946,NaN,NaN,NaN,8.951837,NaN,8.417130,...,NaN,NaN,14.823595,5.756048,NaN,NaN,8.586853,3.03125,NaN,7.289008
1999-12-01,NaN,4.910099,11.044152,23.517157,NaN,NaN,NaN,9.899887,NaN,8.677205,...,NaN,NaN,14.823595,5.827809,NaN,NaN,10.156491,3.06250,NaN,6.766294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-01,21.650000,12.880837,44.797794,48.973061,5.451504,156.380005,97.239998,58.832996,4.617316,29.714472,...,120.980003,13.180000,2.900000,112.078941,29.160000,40.680000,30.804956,5.10000,15.72,244.535355
2024-04-01,20.930000,11.846825,41.641201,48.588249,4.749670,147.940002,86.889999,58.555107,4.604386,30.082024,...,108.760002,10.290000,3.740000,107.472137,29.629999,38.099998,26.741238,4.80000,13.92,235.815094
2024-05-01,22.920000,13.556043,46.703659,54.285213,5.015729,138.479996,88.070000,68.023552,4.515077,42.887547,...,127.900002,14.460000,3.530000,127.833809,31.580000,44.290001,28.079292,5.18000,13.71,231.499710


In [23]:
df_monthly_adj_close.to_csv('../data/10_monthly_adjacent_close.csv')

### Average historical return

In [48]:
df_monthly_adj_close = pd.read_csv('../data/10_monthly_adjacent_close.csv', index_col=0)

#### Converting 'Adj Close' Values to Percentage Change
The "Adj Close" will be transformed to represent the percentage change from the previous day's close. The percentage change will be calculated as 1 + percentage change, which indicates the relative change in the adjusted close values day-over-day.

In [49]:
df_monthly_return = df_monthly_adj_close.copy(deep=True)
for column in df_monthly_return:
    if column != 'Date':
        df_monthly_return[column] = 1 + df_monthly_adj_close[column].pct_change().round(2)
        df_monthly_return[column].dropna()

In [50]:
df_monthly_return.to_csv('../data/10_monthly_returns_complete.csv')

### Return rate

In [51]:
df_monthly_return = pd.read_csv('../data/10_monthly_returns_complete.csv', index_col=0)
df_overview = pd.read_csv('../data/data_5_scaled.csv', index_col=0)

For each ticker, remove the NaN values from the monthly returns. Calculate the return rate by determining the percentage change from the initial listed price to the current price.

In [52]:
df_monthly_adj_close.head()

,KE,SCS,HNI,AVT,ACCO,KEYS,CBRE,BRC,PGRE,BHE,...,ENPH,RUN,SUP,MATX,KIDS,HALO,MATW,KVHI,NEO,UNP
Date,,,,,,,,,,,,,,,,,,,,,
1999-08-01,NaN,6.013282,11.762950,17.141674,NaN,NaN,NaN,8.704611,NaN,13.926089,...,NaN,NaN,15.431546,6.066541,NaN,NaN,9.999043,2.43750,NaN,7.509912
1999-09-01,NaN,5.658068,9.648345,16.270067,NaN,NaN,NaN,9.284917,NaN,13.358641,...,NaN,NaN,15.431546,6.040688,NaN,NaN,11.104928,2.37500,NaN,7.413507
1999-10-01,NaN,5.165832,9.836304,21.209169,NaN,NaN,NaN,8.414455,NaN,6.052765,...,NaN,NaN,14.708194,6.072314,NaN,NaN,9.261786,3.06250,NaN,8.634527
1999-11-01,NaN,5.319273,10.838723,21.354946,NaN,NaN,NaN,8.951837,NaN,8.417130,...,NaN,NaN,14.823595,5.756048,NaN,NaN,8.586853,3.03125,NaN,7.289008
1999-12-01,NaN,4.910099,11.044152,23.517157,NaN,NaN,NaN,9.899887,NaN,8.677205,...,NaN,NaN,14.823595,5.827809,NaN,NaN,10.156491,3.06250,NaN,6.766294


In [53]:
# move Date index to a new column
df_monthly_adj_close = df_monthly_adj_close.reset_index(level=0)

In [54]:
# 1, 5, 10, 25 year returns
# Loop through time spans
for i, years in enumerate(variables.time_span_years):
    # Loop through tickers/stock name
    for j, ticker in enumerate(df_overview['stock_ticker_symbol']):
        # if ticker is found in monthly adjacent columns, meaning there are available data to calculate
        if ticker in df_monthly_adj_close.columns:
            # Get date "years" ago
            date = pd.Timestamp.today() - pd.DateOffset(years=years)
            # Pick only stocks that are after this date
            adj_close_list = df_monthly_adj_close.loc[ pd.to_datetime(df_monthly_adj_close['Date']) >= date, ticker].dropna().tolist()
            if len(adj_close_list) >= 2 & j == 0:
                # update df_overview by inserting return_rate_1y, return_rate_5y, etc
                df_overview.loc[df_overview['stock_ticker_symbol'] == ticker, 'return_rate' + '_' + str(years) + 'y_avg'] = np.mean(adj_close_list)

### Volatility
To calculate the volatility of a stock's monthly returns, there are several steps that should be applied to the historical price data.
* Calculate Monthly Returns
* Calculate the Average Monthly Return
* Calculate the Monthly Return Deviations
* Square the Deviations
* Calculate the Variance
* Calculate the Standard Deviation

In [55]:
# 1, 5, 10, 25 year returns
# Loop through time spans
for i, years in enumerate(variables.time_span_years):
    for i, ticker in enumerate(df_overview['stock_ticker_symbol']):
        if ticker in df_monthly_adj_close.columns:
            # Get date "years" ago
            date = pd.Timestamp.today() - pd.DateOffset(years=years)
            # Pick only stocks that are after this date
            adj_close_filtered = df_monthly_adj_close.loc[ pd.to_datetime(df_monthly_adj_close['Date']) >= date, ticker].dropna()
            std_deviation = adj_close_filtered.pct_change().std()

            if len(adj_close_filtered) >= 2:
                df_overview.loc[df_overview['stock_ticker_symbol'] == ticker, 'volatility' + '_' + str(years) + 'y'] = std_deviation

In [56]:
df_overview[['stock_ticker_symbol', 'return_rate_10y_avg', 'volatility_10y']].sort_values(by=['return_rate_10y_avg', 'volatility_10y'])

,stock_ticker_symbol,return_rate_10y_avg,volatility_10y
740,BIRD,3.098281,0.209238
889,MVIS,3.101176,0.338696
453,POWW,3.383483,0.203108
581,DXLG,3.505966,0.275211
785,FINV,3.876345,0.170065
...,...,...,...
475,CVNA,NaN,0.331642
695,RENT,NaN,0.428427
166,IVT,NaN,1.350272
594,TRTN.PRE,NaN,NaN


Store changes

In [57]:
df_overview.to_csv('../data/data_10_overview.csv')
df_monthly_return.to_csv('../data/data_10_monthly_returns_complete.csv')